# Variational AutoEncoder (MLP) Pytorch
Train a variational autoencoder with MNIST dataset

#### References:
* http://kvfrans.com/variational-autoencoders-explained/
* https://github.com/pytorch/examples/blob/master/mnist/main.py
* https://github.com/pytorch/examples/blob/master/vae/main.py
* http://wiseodd.github.io/techblog/2017/01/24/vae-pytorch/
* https://github.com/ritheshkumar95/pytorch-vqvae
* https://pytorch.org/docs/stable/distributions.html
* https://ermongroup.github.io/cs228-notes/extras/vae/

In [1]:
import os
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torch.distributions.normal import Normal
from torchvision import datasets, transforms
from torchvision.utils import save_image
#from torch.utils.tensorboard import SummaryWriter
batch_size = 128
ZDIMS = 10
num_hidden = 800
epochs = 10
num_classes = 10
print("PyTorch Version: ",torch.__version__)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Device:', device)
num_gpu = torch.cuda.device_count()
print('Number of available GPUs:', num_gpu)

log_interval = 10

PyTorch Version:  1.0.0
Device: cpu
Number of available GPUs: 0


#### Download Data

In [2]:
# Download or load downloaded MNIST dataset
# shuffle data at every epoch
train_loader = torch.utils.data.DataLoader(datasets.MNIST('data', 
    train=True, download=True,transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True)

# Same for test data
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=False, transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=False)

#### Helper Function to convert Labels to One_Hot

In [3]:
def idx2onehot(idx, n=num_classes):

    assert idx.shape[1] == 1
    assert torch.max(idx).item() < n

    onehot = torch.zeros(idx.size(0), n)
    onehot.scatter_(1, idx.data, 1)

    return onehot

#### Define Model

In [4]:
class CondVAE(nn.Module):
    def __init__(self):
        super(CondVAE, self).__init__()

        # Encoder
        self.fc1 = nn.Linear((28*28)+10, num_hidden)
        self.fc21 = nn.Linear(num_hidden, ZDIMS)
        self.fc22 = nn.Linear(num_hidden, ZDIMS)
        
        # Decoder
        self.fc3 = nn.Linear(ZDIMS+10, num_hidden)
        self.fc31 = nn.Linear(num_hidden, num_hidden)
        self.fc4 = nn.Linear(num_hidden, 784)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        # Sample from normal 0 mean unit variance distribution
        eps = torch.randn_like(std)
        # Reparametrization trick
        guessed_z = mu + (eps*std)
        return guessed_z

    def decode(self, z_concat):
        h3 = F.relu(self.fc3(z_concat))
        h31 = F.relu(self.fc31(h3))
        return torch.sigmoid(self.fc4(h31))

    def forward(self, x, y_one_hot):
        # Concatenate image and labels
        x = x.view(-1, 784)
        x = torch.cat((x, y_one_hot), dim=1)
        
        # Encode
        mu, logvar = self.encode(x)
        
        # Reparametrization Trick
        z = self.reparameterize(mu, logvar)
        # Concatenate z with labels
        z = torch.cat((z, y_one_hot), dim=1)
        return self.decode(z), mu, logvar

#### Instantiate Model

In [ ]:
model = CondVAE().to(device)

#### Define Optimizer

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)

#### Define Loss Function

In [ ]:
# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

#### Define Train/Test Functions

In [ ]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, label) in enumerate(train_loader):
        data = data.to(device)
        # convert y into one-hot encoding
        y = idx2onehot(label.view(-1, 1))
        y = y.to(device)
        
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data, y)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))


def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, label) in enumerate(test_loader):
            data = data.to(device)
            # convert y into one-hot encoding
            y = idx2onehot(label.view(-1, 1))
            y = y.to(device)
            recon_batch, mu, logvar = model(data, y)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(batch_size, 1, 28, 28)[:n]])
                save_image(comparison.cpu(),
                         'results/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

#### Train/Test

In [ ]:
# 206 loss
for epoch in range(1, epochs + 1):
        train(epoch)
        test(epoch)
        
        with torch.no_grad():
            z = torch.randn(64, ZDIMS).to(device)
            #y = torch.randint(0, num_classes, (64, 1)).to(dtype=torch.long)
            y = torch.ones(64, 1).to(dtype=torch.long) * 8

            y = idx2onehot(y).to(device, dtype=z.dtype)
            z = torch.cat((z, y), dim=1)

            sample = model.decode(z).cpu()
            save_image(sample.view(64, 1, 28, 28), 'results/sample_' + str(epoch) + '.png')

Train Epoch: 1 [0/60000 (0%)]	Loss: 543.856995
Train Epoch: 1 [1280/60000 (2%)]	Loss: 224.860336
Train Epoch: 1 [2560/60000 (4%)]	Loss: 203.253342
Train Epoch: 1 [3840/60000 (6%)]	Loss: 205.575348
Train Epoch: 1 [5120/60000 (9%)]	Loss: 198.181808
Train Epoch: 1 [6400/60000 (11%)]	Loss: 195.243240
Train Epoch: 1 [7680/60000 (13%)]	Loss: 194.337067
Train Epoch: 1 [8960/60000 (15%)]	Loss: 184.024246
Train Epoch: 1 [10240/60000 (17%)]	Loss: 182.511765
Train Epoch: 1 [11520/60000 (19%)]	Loss: 179.244659
Train Epoch: 1 [12800/60000 (21%)]	Loss: 168.472015
Train Epoch: 1 [14080/60000 (23%)]	Loss: 172.853683
Train Epoch: 1 [15360/60000 (26%)]	Loss: 156.345856
Train Epoch: 1 [16640/60000 (28%)]	Loss: 167.428802
Train Epoch: 1 [17920/60000 (30%)]	Loss: 154.530884
Train Epoch: 1 [19200/60000 (32%)]	Loss: 157.655548
Train Epoch: 1 [20480/60000 (34%)]	Loss: 154.168304
Train Epoch: 1 [21760/60000 (36%)]	Loss: 147.943054
Train Epoch: 1 [23040/60000 (38%)]	Loss: 152.665161
Train Epoch: 1 [24320/60000 

In [ ]:
with torch.no_grad():
            z = torch.randn(64, ZDIMS).to(device)
            #y = torch.randint(0, num_classes, (64, 1)).to(dtype=torch.long)
            y = torch.ones(64, 1).to(dtype=torch.long) * 1

            y = idx2onehot(y).to(device, dtype=z.dtype)
            z = torch.cat((z, y), dim=1)

            sample = model.decode(z).cpu()
            save_image(sample.view(64, 1, 28, 28), 'results/sample_' + str(epoch) + '.png')